In [3]:
import pygame
import numpy
import threading
import pyaudio
import scipy
import scipy.fftpack
import scipy.io.wavfile
import sys
import wave
from IPython.display import clear_output

In [7]:
import sounddevice as sd
sd.query_devices()

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Digidesign Mbox2 Analog 1/2 (2-, MME (2 in, 0 out)
   2 Digidesign Mbox2 S/PDIF 3/4 (2-, MME (2 in, 0 out)
   3 Microphone (HD Pro Webcam C920), MME (2 in, 0 out)
   4 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  5 Speakers (Realtek High Definiti, MME (0 in, 8 out)
   6 Realtek Digital Output (Realtek, MME (0 in, 2 out)
   7 ASUS VP228 (3- NVIDIA High Defi, MME (0 in, 2 out)
   8 ASUS VP228 (3- NVIDIA High Defi, MME (0 in, 2 out)
   9 Digidesign Mbox2 Analog 1/2 (2-, MME (0 in, 2 out)
  10 Digidesign Mbox2 Analog 1/2 (Mbox2 In 1/2 In), Windows WDM-KS (2 in, 0 out)
  11 Digidesign Mbox2 S/PDIF 3/4 (Mbox2 In S/PDIF 3/4 In), Windows WDM-KS (2 in, 0 out)
  12 Digidesign Mbox2 Analog 1/2 (Mbox2 Out 1/2 Out), Windows WDM-KS (0 in, 2 out)
  13 SPDIF Out (Realtek HDA SPDIF Out), Windows WDM-KS (0 in, 2 out)
  14 Speakers (Realtek HD Audio output), Windows WDM-KS (0 in, 8 out)
  15 Stereo Mix (Realtek HD Audio Stereo input), W

In [12]:
SAMPLE_RATE = 44100  # try 5000 for HD data, 48000 for realtime
DEVICE = 1
W = 500
fftsize = 512
currentCol = 0
scooter = []
overlap = 5  # 1 for raw, realtime - 8 or 16 for high-definition

In [13]:
def update_matrix(pcm):
    global data
    ffty = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    ffty = abs(ffty[0:int(len(ffty)/2)])/500  # FFT is mirror-imaged
    # ffty=(scipy.log(ffty))*30-50 # if you want uniform data
    print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    for i in range(len(ffty)):
        if ffty[i] < 0:
            ffty[i] = 0
        if ffty[i] > 255:
            ffty[i] = 255
    scooter.append(ffty)
    if len(scooter) < 6:
        return
    scooter.pop(0)
    ffty = (scooter[0]+scooter[1]*2+scooter[2]*3+scooter[3]*2+scooter[4])/9
    data = numpy.roll(data, -1, 0)
    data[-1] = ffty[::-1]

In [14]:
def record():
    # p = pyaudio.PyAudio()
    # inStream = p.open(format=pyaudio.paInt16,
    #                   channels=1,
    #                   rate=SAMPLE_RATE,
    #                   input_device_index=DEVICE, input=True)
    # while True:
    #     pcm = numpy.frombuffer(inStream.read(fftsize*4))
    #     #print('PCM FIRST LEN',len(pcm))
    #     update_matrix(pcm)
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=SAMPLE_RATE,
                      input_device_index=DEVICE, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[int(fftsize/overlap):]
        pcm = numpy.frombuffer(inStream.read(
            int(fftsize/overlap)), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        update_matrix(linear)

In [ ]:
pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]
print( max(pal), min(pal))
data = numpy.array(numpy.zeros((W, int(fftsize/2))), dtype=int)
# data=Numeric.array(data) # for older PyGame that requires Numeric
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((W, fftsize/2))
world = pygame.Surface((W, fftsize/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
clk = pygame.time.Clock()
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    pygame.surfarray.blit_array(world, data)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

In [13]:

def graphFFT(pcm):
    global currentCol, data
    ffty = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    ffty = abs(ffty[0:int(len(ffty)/2)])/500  # FFT is mirror-imaged
    # ffty=(scipy.log(ffty))*30-50 # if you want uniform data
    print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    clear_output(wait = True)
    for i in range(len(ffty)):
        if ffty[i] < 0:
            ffty[i] = 0
        if ffty[i] > 255:
            ffty[i] = 255
    scooter.append(ffty)
    if len(scooter) < 6:
        return
    scooter.pop(0)
    ffty = (scooter[0]+scooter[1]*2+scooter[2]*3+scooter[3]*2+scooter[4])/9
    data = numpy.roll(data, -1, 0)
    data[-1] = ffty[::-1]
    currentCol += 1
    if currentCol == windowWidth:
        currentCol = 0


def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[int(fftsize/overlap):]
        pcm = numpy.frombuffer(inStream.read(
            int(fftsize/overlap)), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)


pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]
print( max(pal), min(pal))
data = numpy.array(numpy.zeros((windowWidth, int(fftsize/2))), dtype=int)
# data=Numeric.array(data) # for older PyGame that requires Numeric
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((windowWidth, fftsize/2))
world = pygame.Surface((windowWidth, fftsize/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
clk = pygame.time.Clock()
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    pygame.surfarray.blit_array(world, data)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

Exception in thread Thread-11 (record):
Traceback (most recent call last):
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\chomps\AppData\Local\Temp\ipykernel_1308\3651637372.py", line 32, in record
  File "C:\Users\chomps\.conda\envs\audios\Lib\site-packages\pyaudio.py", line 608, in read
    return pa.read_stream(self._stream, num_frames, exception_on_overflow)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno -9981] Input overflowed


SystemExit: 

In [ ]:
soundcard = 1

In [ ]:
def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[fftsize/overlap:]
        pcm = numpy.fromstring(inStream.read(
            fftsize/overlap), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)

In [ ]:
record()

In [18]:
p = pyaudio.PyAudio()
inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=1, input=True)